In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import KFold
import pandas as pd
import os
from PIL import Image

# Dataset 정의
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name)
        label = int(self.annotations.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

# 하이퍼파라미터 설정
num_epochs = 10
num_classes = 17  # 클래스 수에 맞게 변경
batch_size = 16
learning_rate = 0.001
k_folds = 5

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 데이터셋 로드
dataset = CustomDataset(csv_file='cv_challenge/train.csv', root_dir='cv_challenge/train', transform=transform)

# K-Fold Cross Validation 설정
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# 모델 학습
for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Subset 생성
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)

    # 모델, 손실 함수, 옵티마이저 정의
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to('cuda')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 학습
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to('cuda'), labels.to('cuda')

            # forward + backward + optimize
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

    # 검증
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy for fold {fold}: {100 * correct / total}%')

print('Training complete')


FOLD 0
--------------------------------


C:\Users\master\anaconda3\envs\EDA\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\master\anaconda3\envs\EDA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.7510781953040557
Epoch 2, Loss: 0.3251116579940802
Epoch 3, Loss: 0.23754442499691172
Epoch 4, Loss: 0.16004878851950546
Epoch 5, Loss: 0.10782245968621743
Epoch 6, Loss: 0.1531050384716614
Epoch 7, Loss: 0.08833123100744679
Epoch 8, Loss: 0.08363539422567509
Epoch 9, Loss: 0.058457261826725136
Epoch 10, Loss: 0.10825164363236178
Accuracy for fold 0: 83.43949044585987%
FOLD 1
--------------------------------
Epoch 1, Loss: 0.7568633611066432
Epoch 2, Loss: 0.40482761093144176
Epoch 3, Loss: 0.23060211782119697
Epoch 4, Loss: 0.14089188154173804
Epoch 5, Loss: 0.09231429469264761
Epoch 6, Loss: 0.1267648787662104
Epoch 7, Loss: 0.1966799614550192
Epoch 8, Loss: 0.12305308478778298
Epoch 9, Loss: 0.08504936049096994
Epoch 10, Loss: 0.10039955339559554
Accuracy for fold 1: 85.03184713375796%
FOLD 2
--------------------------------
Epoch 1, Loss: 0.7682004017354567
Epoch 2, Loss: 0.36468326191924794
Epoch 3, Loss: 0.275964250634837
Epoch 4, Loss: 0.1948620078825875
Epoch 5

In [5]:
from copy import deepcopy
from PIL import Image
import numpy as np


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# 모델 리스트
models_list = {
    'resnet18': models.resnet18(pretrained=True),
    'vgg16': models.vgg16(pretrained=True),
    'efficientnet_b0': models.efficientnet_b0(pretrained=True)
}

# 가장 높은 정확도를 기록한 모델과 fold 저장
best_accuracy = 0.0
best_model_state = None
best_fold = None
best_model_name = ''
best_fold_train_idx = None
best_fold_val_idx = None

# 모델 학습
for model_name, model in models_list.items():
    print(f'Model: {model_name}')
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Subset 생성
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)

        # 모델, 손실 함수, 옵티마이저 정의
        if model_name == 'resnet18':
            model.fc = nn.Linear(model.fc.in_features, num_classes)
        elif model_name == 'vgg16':
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
        elif model_name == 'efficientnet_b0':
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        model = model.to('cuda')

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # 학습
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to('cuda'), labels.to('cuda')

                # forward + backward + optimize
                outputs = model(images)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

        # 검증
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to('cuda'), labels.to('cuda')
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy for fold {fold}: {accuracy}%')

        # 가장 높은 정확도를 기록한 모델과 fold 저장
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_state = deepcopy(model.state_dict())
            best_fold = fold
            best_model_name = model_name
            best_fold_train_idx = train_idx
            best_fold_val_idx = val_idx

print(f'Best Model: {best_model_name}, Fold: {best_fold}, Accuracy: {best_accuracy}%')

# 가장 높은 정확도를 기록한 모델 저장
torch.save(best_model_state, f'best_model_{best_model_name}_fold_{best_fold}.pth')

# 가장 높은 정확도의 fold 데이터 저장
train_data = dataset.annotations.iloc[best_fold_train_idx]
val_data = dataset.annotations.iloc[best_fold_val_idx]
train_data.to_csv(f'best_train_data_{best_model_name}_fold_{best_fold}.csv', index=False)
val_data.to_csv(f'best_val_data_{best_model_name}_fold_{best_fold}.csv', index=False)

print('Training complete')

Model: resnet18
FOLD 0
--------------------------------
Epoch 1, Loss: 0.7194109515676016
Epoch 2, Loss: 0.2929082589719114
Epoch 3, Loss: 0.2244304210607764
Epoch 4, Loss: 0.1419063522185706
Epoch 5, Loss: 0.12093786279894883
Epoch 6, Loss: 0.10014770448773722
Epoch 7, Loss: 0.1173254294981119
Epoch 8, Loss: 0.11440968116203064
Epoch 9, Loss: 0.09895111957072764
Epoch 10, Loss: 0.0720723496150974
Accuracy for fold 0: 89.171974522293%
FOLD 1
--------------------------------
Epoch 1, Loss: 0.4112218420811092
Epoch 2, Loss: 0.16189258524417122
Epoch 3, Loss: 0.13787862309549428
Epoch 4, Loss: 0.1414782386037368
Epoch 5, Loss: 0.09764549723838138
Epoch 6, Loss: 0.04173720481693509
Epoch 7, Loss: 0.03511175431776792
Epoch 8, Loss: 0.055508983328119155
Epoch 9, Loss: 0.022156072015317062
Epoch 10, Loss: 0.014043463009433184
Accuracy for fold 1: 97.45222929936305%
FOLD 2
--------------------------------
Epoch 1, Loss: 0.33571733207783744
Epoch 2, Loss: 0.04252786183717979
Epoch 3, Loss: 0.03